<a href="https://colab.research.google.com/github/fwitschel/QDMKM/blob/main/notebooks/QDMKM_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import os

# Execute this code only if in colab
if 'COLAB_GPU' in os.environ:
  print("Executing in Colab!")
  # Cloning GitHub repository
  !git clone https://github.com/fwitschel/QDMKM.git
  %cd QDMKM


Executing in Colab!
fatal: destination path 'QDMKM' already exists and is not an empty directory.
/content/QDMKM/QDMKM


We install some libraries that we will need later

In [44]:
!pip install langchain langchain-community pypdf sentence_transformers faiss-cpu langchain-anthropic groq chromadb rank_bm25 llama-index llama-index-retrievers-bm25

We read the input file into a so-called dataframe.

In [45]:
import pandas as pd
emails = pd.read_csv("/content/QDMKM/data/cases-emails.csv")
print(emails)

                 sender                 receiver  \
0     student42@MBFH.ch      my_msc_dean@MBFH.ch   
1     student21@MBFH.ch      my_msc_dean@MBFH.ch   
2     student84@MBFH.ch  former_msc_dean@MBFH.ch   
3    student168@MBFH.ch      my_msc_dean@MBFH.ch   
4    student336@MBFH.ch      my_msc_dean@MBFH.ch   
5    student672@MBFH.ch      my_msc_dean@MBFH.ch   
6     student72@MBFH.ch      my_msc_dean@MBFH.ch   
7     student67@MBFH.ch      my_msc_dean@MBFH.ch   
8      student7@MBFH.ch      my_msc_dean@MBFH.ch   
9    student745@MBFH.ch  former_msc_dean@MBFH.ch   
10    student45@MBFH.ch      my_msc_dean@MBFH.ch   
11    student74@MBFH.ch      my_msc_dean@MBFH.ch   
12     student5@MBFH.ch      my_msc_dean@MBFH.ch   
13   student666@MBFH.ch      my_msc_dean@MBFH.ch   
14   student888@MBFH.ch      my_msc_dean@MBFH.ch   
15  student4242@MBFH.ch         my_msc_dean@MBFH   

                                     subject  first_date  notice_weeks  \
0                        Re: late submiss

In the input file, each case is represented by one row. We create one Document object for each case that contains, as textual content (to be transformed and stored as embedding vectors) the subject of the initial email, followed by the entire text of the conversation. As metadata, we keep the email address of the sender and the notice with which the extension was requested. Later, it can be useful to have quick access to this metadata...

In [46]:
import datetime as dt
from langchain_core.documents import Document
docs = []
for index, row in emails.iterrows():
    sender = row['sender']
    subject = row['subject']
    text = subject + " " + row['all_text']
    notice = row['notice_weeks']
    year = dt.datetime.strptime(row['first_date'], '%Y-%m-%d').year
    document = Document(
        page_content=text,
        metadata={"source": sender,"notice":notice, "year":year}
    )
    docs.append(document)

print(docs[0])

page_content='Re: late submission Dear Ms Smith, unfortunately, we cannot accept your request for deadline extension. Since your sickness occurred during a non-critical period of your thesis work and was comparatively short, there was enough time to resolve issues resulting from it. We are looking forward to receiving your thesis submission on June 21st. Best regards, The Dean. ---- Dear Prof. Dean, please find attached the certificate for my sickness. Hoping for a positive decision, best regards, Jane Smith. ---- Dear Ms Smith, could you please send us a medical certificate for your sick period. Please note that this does not imply that we will grant the extension, it is just a routine request. Thanks and best regards, The Dean. --- Dear Prof. Dean, I am writing to you to ask for a deadline extension of 1 week for my master thesis. In February, I had a really bad flu from which it took me two weeks to recover. I feel that I am still suffering from the consequences since my whole thesi

First, we want to index our emails for basic keyword retrieval. For this, we define a function that will do some pro-processing...

In [47]:
from nltk.stem.snowball import SnowballStemmer
import nltk

nltk.download('stopwords')

def preprocess(text):

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # lowercase the tokens
    tokens = [token.lower() for token in tokens]

    # stemming
    stemmer = SnowballStemmer('german', ignore_stopwords=True)
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Then, we do the actual pre-processing on our documents. We store the pre-processed texts in a list, in the same order that they have in the document collection (thus, we can use the order to identify the metadata that belongs to a pre-processed text)

In [48]:
from rank_bm25 import BM25Okapi
import operator

preprocessed_corpus = []
for doc in docs:
  doc_tokens = preprocess(doc.page_content)
  preprocessed_corpus.append(doc_tokens)

We define the new case for which would like to retrieve and summarize similar historical cases. We also the the number (topk) of most similar cases to consider

In [87]:
topk = 3
query = "A student got a very late feedback regarding his MRTP that he wants to react to."
#query = "A student discovered that she was pregnant soon after starting the thesis proposal. Towards the end of her thesis, the pregancy became complicated and she had to take leave. A sickness certificate is available."
#query = "A student needs more time because he had to take over more responsibilities for a new project / mission. His employer assigned him as a project leader and he could not refuse it."

We build a keyword retrieval object and use it to run our query against all emails and to get a score for each email (hopefully indicating how relevant the email is for the new case)

In [81]:
bm25_retriever = BM25Okapi(preprocessed_corpus)
doc_scores = bm25_retriever.get_scores(preprocess(query))

# retrieve the documents (including metadata) corresponding to the topk highest scores
# by first inserting doc ids and corresponding scores into a dict and then sorting it by
# the values (=scores)
doc_scores_dict = {}
for i in range(len(doc_scores)):
  doc_scores_dict[i] = doc_scores[i]
sorted_scores = sorted(doc_scores_dict.items(), key=operator.itemgetter(1))

After getting the scores, we create a list where we insert pairs of documents (including their metadata) and their scores, only for the topk top-ranked documents...

In [82]:
bm25_results = []
for i in range(topk):
  (doc_id, score) = sorted_scores[len(docs)-i-1]
  cur_doc = docs[doc_id]
  bm25_results.append((cur_doc, score))

for result in bm25_results:
  print(result)

(Document(metadata={'source': 'student672@MBFH.ch', 'notice': 8.0, 'year': 2022}, page_content="Re:my thesis Dear Ms Big, yes we can extend your submission by 2 weeks, given the very unfortunate and exceptional circumstances that you could not foresee. Please make sure to submit your thesis until August 4th, midnight. Regards, The Dean ---- Dear Dean, I'm sorry that I have to trouble you with this, but I have had a lot of unexpected trouble in the last months which prevents me from submitting my thesis on time. Last year in October, our little baby boy was born. We had planned everything around that (my thesis plan had included a two-month break and we had organised a day care as well as a break for my partner between January and March). But then, my mother got very sick and since she is alone, I had to go home and take care of her. I include the medical certificate for her sickness. I tried to catch up all the work after my return, but it was too much for the few hours that remain eac

Now, we take an embeddings model and use it to create embeddings vectors for our email conversations. In the end, such an embedding vector is just a bunch of numbers...

In [83]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

chunk_texts = list(map(lambda d: d.page_content, docs))
embeddings = bge_embeddings.embed_documents(chunk_texts)
print(embeddings[0])

[-0.06024385988712311, 0.026430662721395493, 0.022581422701478004, 0.013069221749901772, 0.017530471086502075, -0.002363329753279686, 0.030586568638682365, 0.028281182050704956, -0.023568671196699142, -0.02540016919374466, 0.01419154740869999, -0.02738691121339798, -0.005180547013878822, -0.009107926860451698, 0.028404181823134422, 0.01528555154800415, 0.004506260622292757, -0.00012868685007560998, -0.011500484310090542, 0.05693953484296799, -0.011474189348518848, -0.0036948933266103268, -0.004475072957575321, -0.018138622865080833, -0.028383374214172363, -0.012660247273743153, -0.002087835455313325, -0.03936571627855301, -0.026432760059833527, -0.19615943729877472, -0.04876246303319931, -0.02333776094019413, -0.0004094137402717024, -0.023912718519568443, 0.024079622700810432, -0.007743343245238066, -0.024135878309607506, 0.02551259659230709, -0.02752256765961647, 0.03140414506196976, 0.04504133015871048, 0.02439050003886223, -0.03522006422281265, -0.031253378838300705, -0.011611488647

In [84]:
from langchain_community.vectorstores import FAISS

text_embedding_pairs = zip(chunk_texts, embeddings)
db = FAISS.from_embeddings(text_embedding_pairs, bge_embeddings)

Then we retrieve the topk most similar cases using semantic search, i.e. comparison of embeddings.

In [88]:
contexts = db.similarity_search_with_score(query, k=topk)

for i in range(topk):
  print(contexts[i])

(Document(id='75d30b28-a520-40e9-a33c-6588946c3db0', metadata={}, page_content='Re: late submission Dear Ms Smith, unfortunately, we cannot accept your request for deadline extension. Since your sickness occurred during a non-critical period of your thesis work and was comparatively short, there was enough time to resolve issues resulting from it. We are looking forward to receiving your thesis submission on June 21st. Best regards, The Dean. ---- Dear Prof. Dean, please find attached the certificate for my sickness. Hoping for a positive decision, best regards, Jane Smith. ---- Dear Ms Smith, could you please send us a medical certificate for your sick period. Please note that this does not imply that we will grant the extension, it is just a routine request. Thanks and best regards, The Dean. --- Dear Prof. Dean, I am writing to you to ask for a deadline extension of 1 week for my master thesis. In February, I had a really bad flu from which it took me two weeks to recover. I feel th

Here, we connect to an LLM at Groq. To make it work, please get yourself an API key for GROQ and store it as a key on the left side of this notebook...!

In [58]:
from groq import Groq
def llm(groq_client, prompt):
  chat_completion = groq_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama-3.3-70b-versatile",
  )

  return chat_completion.choices[0].message.content

In [59]:
groq_client = Groq(
    api_key=userdata.get('GROQ_API_KEY')
)

Here, you instruct the Large Language Model what to do:

* In the "system" part of the prompt, you explain the general task, including the
context (i.e. the retrieved information) that the system should rely on. You can pass the content of the retrieved emails by putting "{context}" into this part of the prompt
* In the "query" part of the prompt, you give instruction to make a decision about the new case (as introduced already above, before the retrieval)

In [60]:
context = '\n\n'.join(list(map(lambda c: c[0].page_content, semantic_results)))
prompt = f"""You are a helpful assistant that helps a study dean to decide about students' request for extending the deadline of their master theses.
        The current case is described as follows: {query}.
        To decide about the current case, the following historical emails seem to be relevant: {context}. Please make a suggestion whether or not
        to grant the deadline extension, including a justification that is based on the given context! If possible, please include quotes from the historical emails"""
print(llm(groq_client, prompt))

Based on the given context, I would suggest granting the deadline extension. The student has received late feedback regarding their MRTP and wants to react to it, which is a reasonable request. As the student mentions in their email, "I am really desperate: because my supervisor was sick for a long time in winter, I did not receive an MTRP assessment until now. Now, I got one and there is a point that I should correct that my supervisor sees as essential and I do not have the time to work it out now."

The dean's previous response to a similar request was "while this is very unfortunate, it is by no means certain that the consequences that you fear will arise. Please talk to your supervisor and clarify how to solve this. Without a feedback from her, I am not willing to grant an extension." However, in this case, the student has already received feedback from their supervisor, which identifies a point that needs to be corrected. The student is not simply speculating about potential cons